# ML-Fundamentals - Neural Networks - Exercise: Neural Network Framework

# Tasks
Your main goal is to extend the existing framework, to perform experiments with different model combinations and to document your observations. Here is a list of necessary tasks and some ideas for additional points:
  * (6) Write a data loader for a different image dataset, e.g., CIFAR or Labelled Faces in the Wild. Feel free to search a dataset you like to classify. Create and train a simple fully connected network on that dataset in this notebook.
  * (10) Implement the `Conv` and `Pool` Layer in `layer.py`. Create and train a convolutional neural network on Mnist and your chosen dataset in this notebook.

Bonus points
  * (5) 1 to 5 points are given for improving the class and method comments in the framework files. Points are given based on the quality and quantity of the comments.
  * (1) For each additional implemented activation functions in `activation_func.py` you get 1 bonus point (max 4 points). Test your implementation in this notebook and observe effects on your networks. Keep an eye on your layer initialization.
  * (2) Implement `Dropout` in `layer.py` and test your implementation with a toy example. Create and train a model that includes Dropout as a layer.
  * (5) Implement `Batchnorm` in `layer.py` and test your implementation with a toy example. Create and train a model that includes Dropout as a layer.
  * (4) Implement another optimization algorithm in `optimizer.py`. Train one of your models with that new optimizer.
  * (5) Do something extra, up to 5 points.  
  
Please document thoroughly and explain what you do in your experiments, so that work in the notebook is comprehensible, else no points are given.

Fragen
* Sollen wir nur für Pool Layer nur Max forward und backward implementieren? Falls nein, wie solen wir das mit der backward implementierung machen?
* Siehe in layers.py: 
    * In Pooling.backward: What is if not only one pixel has the max value?
    * In Conv Layer: soll zu jedem Pixel bias hinzugefügt werden oder erst nach sum?
    * In Dropout: maske für alle images gleich oder für jedes eine andere? Und wie sollen wir damit umgehen, wenn wir vorhersagen wollen?
    * Braucht man nach Dropout oder Pool Layer eine Aktivierungsfunktion?
    * Pooling Layer gibt immer gleichen loss zurück, woran kann das liegen?

# Requirements

## Python-Modules

In [4]:
%load_ext autoreload
%autoreload 2
# custom 
from htw_nn_framework.networks import NeuralNetwork
from htw_nn_framework.layer import *
from htw_nn_framework.activation_func import *
from htw_nn_framework.loss_func import *
from htw_nn_framework.optimizer import *

# third party
from deep_teaching_commons.data.fundamentals.mnist import Mnist

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

In [5]:
# create mnist loader from deep_teaching_commons
mnist_loader = Mnist(data_dir='data')

# load all data, labels are one-hot-encoded, images are flatten and pixel squashed between [0,1]
train_images, train_labels, test_images, test_labels = mnist_loader.get_all_data(flatten=False, one_hot_enc=False, normalized=True)
print(train_images.shape, train_labels.shape)

# reshape to match generell framework architecture 
train_images, test_images = train_images.reshape(60000, 1, 28, 28), test_images.reshape(10000, 1, 28, 28)            
print(train_images.shape, train_labels.shape)

# shuffle training data
shuffle_index = np.random.permutation(60000)
train_images, train_labels = train_images[shuffle_index], train_labels[shuffle_index]

auto download is active, attempting download
mnist data directory already exists, download aborted
(60000, 28, 28) (60000,)
(60000, 1, 28, 28) (60000,)


# MNIST Fully Connected Network Example
This model and optimization is taken from `framework_exercise.ipynb` as an example for a typical pipeline using the framework files.

In [31]:
train_images_batch, train_labels_batch, test_images_batch, test_labels_batch = train_images[:100,:,:,:], train_labels[:100], test_images[:100,:,:,:], test_labels[:100]

In [ ]:
# design a three hidden layer architecture with Dense-Layer
# and ReLU as activation function
def fcn_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(784, 500)
    relu_01 = ReLU()
    hidden_02 = FullyConnected(500, 200)
    relu_02 = ReLU()
    hidden_03 = FullyConnected(200, 100)
    relu_03 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, relu_02, hidden_03, relu_03, ouput]

# create a neural network on specified architecture with softmax as score function
fcn = NeuralNetwork(fcn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn = Optimizer.sgd(fcn, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

# Todo: Your Extensions and Experiments

In [26]:
train_images_batch, train_labels_batch, test_images_batch, test_labels_batch = train_images[:100,:,:,:], train_labels[:100], test_images[:100,:,:,:], test_labels[:100]

In [35]:
# design a one hidden layer architecture with Conv-Layer
# and ReLU as activation function
def cnn_mnist():
    conv = Conv(train_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    flat = Flatten()
    ouput = FullyConnected(784, 10)
    return [conv, relu_01, flat, ouput]

# create a neural network on specified architecture with softmax as score function
cnn = NeuralNetwork(cnn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn = Optimizer.sgd(cnn, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

Epoch 1
Loss = 2.2694578452448795 :: Training = 0.45 :: Test = 0.26
Epoch 2
Loss = 1.9110738698244194 :: Training = 0.41 :: Test = 0.25
Epoch 3
Loss = 1.975202062034818 :: Training = 0.65 :: Test = 0.5
Epoch 4
Loss = 1.2662140191974134 :: Training = 0.78 :: Test = 0.59
Epoch 5
Loss = 0.9367188910278469 :: Training = 0.93 :: Test = 0.7
Epoch 6
Loss = 0.5372093838818499 :: Training = 0.91 :: Test = 0.66
Epoch 7
Loss = 0.2647844352773687 :: Training = 1.0 :: Test = 0.72
Epoch 8
Loss = 0.18068986459120123 :: Training = 1.0 :: Test = 0.76
Epoch 9
Loss = 0.0739676511032567 :: Training = 1.0 :: Test = 0.71
Epoch 10
Loss = 0.04715890340149023 :: Training = 1.0 :: Test = 0.71


In [44]:
# design a two hidden layer architecture with Conv-Layer and Pooling-Layer
# and ReLU as activation function
def cnn_pool_mnist():
    conv = Conv(train_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    pool = Pool(train_images_batch.shape, np.max, 2, 1)
    flat = Flatten()
    ouput = FullyConnected(729, 10)
    return [conv, relu_01, pool, flat, ouput]

# create a neural network on specified architecture with softmax as score function
cnn_pool = NeuralNetwork(cnn_pool_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn_pool = Optimizer.sgd(cnn_pool, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

Epoch 1
Loss = 8.349133883558634 :: Training = 0.1 :: Test = 0.14
Epoch 2
Loss = 2.30871212199084 :: Training = 0.13 :: Test = 0.14
Epoch 3
Loss = 2.2965014229676015 :: Training = 0.14 :: Test = 0.08
Epoch 4
Loss = 2.286223601912051 :: Training = 0.14 :: Test = 0.08
Epoch 5
Loss = 2.2775862840431835 :: Training = 0.14 :: Test = 0.08
Epoch 6
Loss = 2.2703342842847514 :: Training = 0.14 :: Test = 0.08
Epoch 7
Loss = 2.264246738982798 :: Training = 0.14 :: Test = 0.08
Epoch 8
Loss = 2.2591340812048846 :: Training = 0.14 :: Test = 0.08
Epoch 9
Loss = 2.254834911933597 :: Training = 0.14 :: Test = 0.08
Epoch 10
Loss = 2.2512128458568714 :: Training = 0.14 :: Test = 0.08


In [34]:
# design a three hidden layer architecture with Dense-Layer and Dropout-Layer
# and ReLU as activation function
def fcn_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(784, 500)
    relu_01 = ReLU()
    hidden_02 = Dropout()
    hidden_03 = FullyConnected(500, 100)
    relu_02 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, hidden_03, relu_02, ouput]

# create a neural network on specified architecture with softmax as score function
fcn = NeuralNetwork(fcn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn = Optimizer.sgd(fcn, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

Epoch 1
Loss = 2.241838426028755 :: Training = 0.31 :: Test = 0.17
Epoch 2
Loss = 2.0186859224479807 :: Training = 0.47 :: Test = 0.19
Epoch 3
Loss = 1.8034993047685224 :: Training = 0.5 :: Test = 0.3
Epoch 4
Loss = 1.5847811386312407 :: Training = 0.58 :: Test = 0.55
Epoch 5
Loss = 1.3640040506297673 :: Training = 0.78 :: Test = 0.62
Epoch 6
Loss = 1.2841362981274649 :: Training = 0.75 :: Test = 0.64
Epoch 7
Loss = 1.0457346896770525 :: Training = 0.82 :: Test = 0.7
Epoch 8
Loss = 0.9075676813155142 :: Training = 0.88 :: Test = 0.67
Epoch 9
Loss = 0.7178575746432586 :: Training = 0.94 :: Test = 0.68
Epoch 10
Loss = 0.6178313401247839 :: Training = 0.94 :: Test = 0.72
